In [2]:
# This is a sample Python script.
import pandas as pd
import torch
device = torch.device('cpu')
import dgl
import networkx as nx
import matplotlib.pyplot as plt
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch.nn as nn
import torch.optim as optim
import dgl.function as fn


In [3]:
df=pd.read_csv("../data/PaySim_kaggle.csv")

df=df.sample(n=5000000)

print(df.head())

         step      type     amount     nameOrig  oldbalanceOrg  \
48459       9   PAYMENT    8517.09  C1544276689        34047.0   
2725634   211  CASH_OUT  123172.45   C262696422        71135.0   
1989810   179  CASH_OUT   67226.13  C1226687628            0.0   
3465072   257   PAYMENT   11726.26   C115245548            0.0   
5733741   399   CASH_IN  184629.59   C337768760        24650.0   

         newbalanceOrig     nameDest  oldbalanceDest  newbalanceDest  isFraud  \
48459          25529.91  M1412197367            0.00            0.00        0   
2725634            0.00  C1748139871      7574089.96      7697262.41        0   
1989810            0.00  C1780779622       269926.46       337152.60        0   
3465072            0.00   M379507303            0.00            0.00        0   
5733741       209279.59   C544985868       262663.25        78033.66        0   

         isFlaggedFraud  
48459                 0  
2725634               0  
1989810               0  
3465072     

In [4]:
# Set the proportion of non-fraud to remove (e.g., 50% of non-fraud instances)
remove_fraction = 0.999

# Separate the fraud and non-fraud instances
fraud_df = df[df['isFraud'] == 1]
non_fraud_df = df[df['isFraud'] == 0]

# Randomly sample and remove 'remove_fraction' proportion of non-fraud instances
non_fraud_to_remove = non_fraud_df.sample(frac=remove_fraction, random_state=42)

# Drop the sampled non-fraud instances from the DataFrame
df = df.drop(non_fraud_to_remove.index)

# Verify the new balance
label_counts = df['isFraud'].value_counts()
fraud_ratio = label_counts[1] / len(df)
non_fraud_ratio = label_counts[0] / len(df)

print(f"Fraud count: {label_counts[1]}")
print(f"Non-fraud count: {label_counts[0]}")
print(f"Fraud ratio: {fraud_ratio:.4f}")
print(f"Non-fraud ratio: {non_fraud_ratio:.4f}")


Fraud count: 6479
Non-fraud count: 4994
Fraud ratio: 0.5647
Non-fraud ratio: 0.4353


In [5]:
# Create a mapping from unique user names to numeric IDs (nodes)
user_mapping = {user: idx for idx, user in enumerate(set(df['nameOrig']).union(set(df['nameDest'])))}

# Create edges between nameOrig and nameDest
src = df['nameOrig'].map(user_mapping).values
dst = df['nameDest'].map(user_mapping).values

In [6]:
print(src)

[15207 18883  4361 ... 22414 17698  7701]


In [7]:
# Create a DGL graph from the source and destination nodes
g = dgl.graph((src, dst))


In [8]:
# Add transaction amount as edge feature
g.edata['amount'] = torch.tensor(df['amount'].values, dtype=torch.float32)

# Optional: Add fraud information to edge features
g.edata['isFraud'] = torch.tensor(df['isFraud'].values, dtype=torch.float32)

# Initialize node features with zeros (this handles all nodes)
num_nodes = g.num_nodes()
balance_orig = torch.zeros(num_nodes, dtype=torch.float32)
balance_dest = torch.zeros(num_nodes, dtype=torch.float32)

for orig_user, balance in df[['nameOrig', 'oldbalanceOrg']].drop_duplicates().values:
    balance_orig[user_mapping[orig_user]] = balance

for dest_user, balance in df[['nameDest', 'oldbalanceDest']].drop_duplicates().values:
    balance_dest[user_mapping[dest_user]] = balance

node_features = torch.stack([balance_orig, balance_dest], dim=1)  # Changed to stack both features

g.ndata['features'] = node_features


In [9]:
g.edata['isFraud'].shape
#node_features.shape
g.num_edges

<bound method DGLGraph.num_edges of Graph(num_nodes=22859, num_edges=11473,
      ndata_schemes={'features': Scheme(shape=(2,), dtype=torch.float32)}
      edata_schemes={'amount': Scheme(shape=(), dtype=torch.float32), 'isFraud': Scheme(shape=(), dtype=torch.float32)})>

In [10]:
# Print graph information
#print(g)

# Visualize the graph using NetworkX (convert DGL graph to NetworkX for visualization)
#nx_graph = g.to_networkx()

# Optional: Visualize using a layout for better readability
#pos = nx.spring_layout(nx_graph)  # Use a layout for better visualization
#plt.figure(figsize=(12, 12))
#nx.draw(nx_graph, pos, node_size=50, node_color='skyblue', font_size=10, with_labels=True)
#plt.show()

In [11]:
# Assuming that you have edge features like transaction amounts or fraud status
edge_features = g.edata.get('amount', None)  # Assuming 'amount' is an edge feature
if edge_features is not None:
    print("Edge Features (Transaction Amounts):")
    print(edge_features)
else:
    print("No edge features found.")


Edge Features (Transaction Amounts):
tensor([3.3740e+01, 1.3933e+05, 2.9360e+06,  ..., 2.3375e+05, 6.5529e+05,
        3.3155e+03])


In [12]:
edges = g.edges()
# Split the dataset into train and test set
# Generate indices for train-test split (80% train, 20% test)
num_train_edges = int(0.8 * len(edges[0]))
train_indices = torch.arange(num_train_edges)
test_indices = torch.arange(num_train_edges, len(edges[0]))

# Create masks for training and testing
train_mask = torch.zeros(len(edges[0]), dtype=torch.bool)
test_mask = torch.zeros(len(edges[0]), dtype=torch.bool)
train_mask[train_indices] = 1
test_mask[test_indices] = 1


In [13]:
# Define the GraphSAGE model for fraud detection
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(GraphSAGE, self).__init__()
        self.layer1 = dgl.nn.SAGEConv(in_feats, hidden_feats, 'mean')
        self.layer2 = dgl.nn.SAGEConv(hidden_feats, out_feats, 'mean')
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(out_feats * 2, 1)  # * Concatenate source and destination node embeddings *
        
    def forward(self, g, features):
        # Apply first GraphSAGE layer and ReLU
        x = self.layer1(g, features)
        x = torch.relu(x)
        x = self.dropout(x)

        # Apply second GraphSAGE layer to get embeddings
        x = self.layer2(g, x)
        
        # Get source and destination nodes for each edge
        src, dst = g.edges()  # Get indices of source and destination nodes
        src_embeddings = x[src]  # Embeddings for source nodes
        dst_embeddings = x[dst]  # Embeddings for destination nodes
        
        # Concatenate source and destination node embeddings to create edge features
        edge_features = torch.cat([src_embeddings, dst_embeddings], dim=1)  # Concatenate along the feature dimension
        logits = self.fc(edge_features).squeeze()  # * Output a single value per edge (fraud score) *
        return logits


In [14]:
# Count occurrences of each label in the 'isFraud' column
label_counts = df['isFraud'].value_counts()

# Calculate the proportion of each class
fraud_ratio = label_counts[1] / len(df)  # Assuming '1' represents fraud
non_fraud_ratio = label_counts[0] / len(df)  # Assuming '0' represents non-fraud

# Print the results
print(f"Fraud count: {label_counts[1]}")
print(f"Non-fraud count: {label_counts[0]}")
print(f"Fraud ratio: {fraud_ratio:.4f}")
print(f"Non-fraud ratio: {non_fraud_ratio:.4f}")

Fraud count: 6479
Non-fraud count: 4994
Fraud ratio: 0.5647
Non-fraud ratio: 0.4353


In [15]:
# Instantiate the model
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
in_feats = 2  # balanceOrig and balanceDest features
hidden_feats = 64
out_feats = 1  # Fraud (binary classification)

model = GraphSAGE(in_feats, hidden_feats, out_feats)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
epochs = 20
for epoch in range(epochs):
    model.train()

    # Forward pass
    logits = model(g, g.ndata['features'])
    print(logits.shape)

    # Get target labels for the fraud detection task
    labels = g.edata['isFraud']
    breakpoint()
    # Compute loss (use train_mask to filter out test edges)
    labels[train_mask]

    logits[train_mask]
    loss = loss_fn(logits[train_mask], labels[train_mask])
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every few epochs
    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

# Evaluate the model
model.eval()
with torch.no_grad():
    logits = model(g, g.ndata['features'])

    # Convert logits to probabilities
    predictions = torch.sigmoid(logits).squeeze()
    
    # Apply threshold of 0.5 to classify fraud
    predicted_labels = (predictions > 0.5).float()
    predicted_labels = predicted_labels[test_mask]  # Apply test_mask here
    # Get actual labels
    #true_labels = g.edata['isFraud']
    true_labels = g.edata['isFraud'][test_mask]  # Apply test_mask here
    # Compute accuracy manually
    correct = (predicted_labels == true_labels).sum().item()
    total = true_labels.size(0)
    accuracy = correct / total

    # Print evaluation results
    print(f"Accuracy: {accuracy:.4f}")
    # Calculate F1-Score
    f1 = f1_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)

    print(f"F1 Score: {f1:.4f}")

torch.Size([11473])
torch.Size([11473])
Epoch 2/20, Loss: 94931.5625
torch.Size([11473])
torch.Size([11473])
Epoch 4/20, Loss: 64249.4921875
torch.Size([11473])
torch.Size([11473])
Epoch 6/20, Loss: 63201.98046875
torch.Size([11473])
torch.Size([11473])
Epoch 8/20, Loss: 55559.6875
torch.Size([11473])
torch.Size([11473])
Epoch 10/20, Loss: 43335.390625
torch.Size([11473])
torch.Size([11473])
Epoch 12/20, Loss: 37003.17578125
torch.Size([11473])
torch.Size([11473])
Epoch 14/20, Loss: 28193.98046875
torch.Size([11473])
torch.Size([11473])
Epoch 16/20, Loss: 22054.80859375
torch.Size([11473])
torch.Size([11473])
Epoch 18/20, Loss: 18774.625
torch.Size([11473])
torch.Size([11473])
Epoch 20/20, Loss: 18044.1015625
Accuracy: 0.6118


NameError: name 'predicte_labels' is not defined